### Childhood Autistic Spectrum Disorder Screening using Machine Learning

The early diagnosis of neurodevelopment disorders can improve treatment and significantly decrease the associated 
healthcare costs. In this project, we will use supervised learning to diagnose Autistic Spectrum Disorder 
(ASD) based on behavioural features and individual characteristics. More specifically, we will build and deploy a neural network using the Keras API. 

This project will use a dataset provided by the UCI Machine Learning Repository that contains screening data for 292 patients. The dataset can be found at the following URL: 
https://archive.ics.uci.edu/ml/datasets/Autistic+Spectrum+Disorder+Screening+Data+for+Children++

Let's dive right in! First, we will import a few of libraries we will use in this project. 

In [ ]:
import sys
import pandas as pd
import sklearn
import keras

print 'Python: {}'.format(sys.version)
print 'Pandas: {}'.format(pd.__version__)
print 'Sklearn: {}'.format(sklearn.__version__)
print 'Keras: {}'.format(keras.__version__)

### 1. Importing the Dataset

We will obtain the data from the UCI Machine Learning Repository; however, since the data isn't contained in a csv or txt file, we will have to download the compressed zip file and then extract the data manually. Once that is accomplished, we will read the information in from a text file using Pandas. 

In [2]:
# import the dataset
file = 'C:/users/brend/tutorial/autism-data.txt'

# read the csv
data = pd.read_table(file, sep = ',', index_col = None)

In [3]:
# print the shape of the DataFrame, so we can see how many examples we have
print 'Shape of DataFrame: {}'.format(data.shape)
print data.loc[0]

Shape of DataFrame: (292, 21)
A1_Score                            1
A2_Score                            1
A3_Score                            0
A4_Score                            0
A5_Score                            1
A6_Score                            1
A7_Score                            0
A8_Score                            1
A9_Score                            0
A10_Score                           0
age                                 6
gender                              m
ethnicity                      Others
jundice                            no
family_history_of_PDD              no
contry_of_res                  Jordan
used_app_before                    no
result                              5
age_desc                 '4-11 years'
relation                       Parent
class                              NO
Name: 0, dtype: object


In [4]:
# print out multiple patients at the same time
data.loc[:10]

A1_Score  A2_Score  A3_Score  A4_Score  A5_Score  A6_Score  A7_Score  \
0          1         1         0         0         1         1         0   
1          1         1         0         0         1         1         0   
2          1         1         0         0         0         1         1   
3          0         1         0         0         1         1         0   
4          1         1         1         1         1         1         1   
5          0         0         1         0         1         1         0   
6          1         0         1         1         1         1         0   
7          1         1         1         1         1         1         1   
8          1         1         1         1         1         1         1   
9          0         0         1         1         1         0         1   
10         1         0         0         0         1         1         1   

    A8_Score  A9_Score  A10_Score  ...  gender          ethnicity jundice  \
0          1         0          0  ...       m             Others      no   
1          1         0          0  ...       m  'Middle Eastern '      no   
2          1         0          0  ...       m                  ?      no   
3          0         0          1  ...       f                  ?     yes   
4          1         1          1  ...       m             Others     yes   
5          1         0          1  ...       m                  ?      no   
6          1         0          1  ...       m     White-European      no   
7          1         0          0  ...       f  'Middle Eastern '      no   
8          0         0          0  ...       f  'Middle Eastern '      no   
9          1         0          0  ...       f                  ?      no   
10         1         1          1  ...       m     White-European     yes   

   family_history_of_PDD     contry_of_res used_app_before result  \
0                     no            Jordan              no      5   
1                     no            Jordan              no      5   
2                     no            Jordan             yes      5   
3                     no            Jordan              no      4   
4                     no   'United States'              no     10   
5                    yes             Egypt              no      5   
6                     no  'United Kingdom'              no      7   
7                     no           Bahrain              no      8   
8                     no           Bahrain              no      7   
9                    yes           Austria              no      5   
10                    no  'United Kingdom'              no      7   

        age_desc relation class  
0   '4-11 years'   Parent    NO  
1   '4-11 years'   Parent    NO  
2   '4-11 years'        ?    NO  
3   '4-11 years'        ?    NO  
4   '4-11 years'   Parent   YES  
5   '4-11 years'        ?    NO  
6   '4-11 years'   Parent   YES  
7   '4-11 years'   Parent   YES  
8   '4-11 years'   Parent   YES  
9   '4-11 years'        ?    NO  
10  '4-11 years'     Self   YES  

[11 rows x 21 columns]

In [5]:
# print out a description of the dataframe
data.describe()

A1_Score    A2_Score    A3_Score    A4_Score    A5_Score    A6_Score  \
count  292.000000  292.000000  292.000000  292.000000  292.000000  292.000000   
mean     0.633562    0.534247    0.743151    0.551370    0.743151    0.712329   
std      0.482658    0.499682    0.437646    0.498208    0.437646    0.453454   
min      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
25%      0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
50%      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   
75%      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   
max      1.000000    1.000000    1.000000    1.000000    1.000000    1.000000   

         A7_Score    A8_Score    A9_Score   A10_Score      result  
count  292.000000  292.000000  292.000000  292.000000  292.000000  
mean     0.606164    0.496575    0.493151    0.726027    6.239726  
std      0.489438    0.500847    0.500811    0.446761    2.284882  
min      0.000000    0.000000    0.000000    0.000000    0.000000  
25%      0.000000    0.000000    0.000000    0.000000    5.000000  
50%      1.000000    0.000000    0.000000    1.000000    6.000000  
75%      1.000000    1.000000    1.000000    1.000000    8.000000  
max      1.000000    1.000000    1.000000    1.000000   10.000000

### 2. Data Preprocessing

This dataset is going to require multiple preprocessing steps. First, we have columns in our DataFrame (attributes) that we don't want to use when training our neural network. We will drop these columns first. Secondly, much of our data is reported using strings; as a result, we will convert our data to categorical labels. During our preprocessing, we will also split the dataset into X and Y datasets, where X has all of the attributes we want to use for prediction and Y has the class labels. 

In [6]:
# drop unwanted columns
data = data.drop(['result', 'age_desc'], axis=1)

In [7]:
data.loc[:10]

A1_Score  A2_Score  A3_Score  A4_Score  A5_Score  A6_Score  A7_Score  \
0          1         1         0         0         1         1         0   
1          1         1         0         0         1         1         0   
2          1         1         0         0         0         1         1   
3          0         1         0         0         1         1         0   
4          1         1         1         1         1         1         1   
5          0         0         1         0         1         1         0   
6          1         0         1         1         1         1         0   
7          1         1         1         1         1         1         1   
8          1         1         1         1         1         1         1   
9          0         0         1         1         1         0         1   
10         1         0         0         0         1         1         1   

    A8_Score  A9_Score  A10_Score age gender          ethnicity jundice  \
0          1         0          0   6      m             Others      no   
1          1         0          0   6      m  'Middle Eastern '      no   
2          1         0          0   6      m                  ?      no   
3          0         0          1   5      f                  ?     yes   
4          1         1          1   5      m             Others     yes   
5          1         0          1   4      m                  ?      no   
6          1         0          1   5      m     White-European      no   
7          1         0          0   5      f  'Middle Eastern '      no   
8          0         0          0  11      f  'Middle Eastern '      no   
9          1         0          0  11      f                  ?      no   
10         1         1          1  10      m     White-European     yes   

   family_history_of_PDD     contry_of_res used_app_before relation class  
0                     no            Jordan              no   Parent    NO  
1                     no            Jordan              no   Parent    NO  
2                     no            Jordan             yes        ?    NO  
3                     no            Jordan              no        ?    NO  
4                     no   'United States'              no   Parent   YES  
5                    yes             Egypt              no        ?    NO  
6                     no  'United Kingdom'              no   Parent   YES  
7                     no           Bahrain              no   Parent   YES  
8                     no           Bahrain              no   Parent   YES  
9                    yes           Austria              no        ?    NO  
10                    no  'United Kingdom'              no     Self   YES

In [8]:
# create X and Y datasets for training
x = data.drop(['class'], 1)
y = data['class']

In [9]:
x.loc[:10]

A1_Score  A2_Score  A3_Score  A4_Score  A5_Score  A6_Score  A7_Score  \
0          1         1         0         0         1         1         0   
1          1         1         0         0         1         1         0   
2          1         1         0         0         0         1         1   
3          0         1         0         0         1         1         0   
4          1         1         1         1         1         1         1   
5          0         0         1         0         1         1         0   
6          1         0         1         1         1         1         0   
7          1         1         1         1         1         1         1   
8          1         1         1         1         1         1         1   
9          0         0         1         1         1         0         1   
10         1         0         0         0         1         1         1   

    A8_Score  A9_Score  A10_Score age gender          ethnicity jundice  \
0          1         0          0   6      m             Others      no   
1          1         0          0   6      m  'Middle Eastern '      no   
2          1         0          0   6      m                  ?      no   
3          0         0          1   5      f                  ?     yes   
4          1         1          1   5      m             Others     yes   
5          1         0          1   4      m                  ?      no   
6          1         0          1   5      m     White-European      no   
7          1         0          0   5      f  'Middle Eastern '      no   
8          0         0          0  11      f  'Middle Eastern '      no   
9          1         0          0  11      f                  ?      no   
10         1         1          1  10      m     White-European     yes   

   family_history_of_PDD     contry_of_res used_app_before relation  
0                     no            Jordan              no   Parent  
1                     no            Jordan              no   Parent  
2                     no            Jordan             yes        ?  
3                     no            Jordan              no        ?  
4                     no   'United States'              no   Parent  
5                    yes             Egypt              no        ?  
6                     no  'United Kingdom'              no   Parent  
7                     no           Bahrain              no   Parent  
8                     no           Bahrain              no   Parent  
9                    yes           Austria              no        ?  
10                    no  'United Kingdom'              no     Self

In [10]:
# convert the data to categorical values - one-hot-encoded vectors
X = pd.get_dummies(x)

In [11]:
# print the new categorical column labels
X.columns.values

array(['A1_Score', 'A2_Score', 'A3_Score', 'A4_Score', 'A5_Score',
       'A6_Score', 'A7_Score', 'A8_Score', 'A9_Score', 'A10_Score',
       'age_10', 'age_11', 'age_4', 'age_5', 'age_6', 'age_7', 'age_8',
       'age_9', 'age_?', 'gender_f', 'gender_m',
       "ethnicity_'Middle Eastern '", "ethnicity_'South Asian'",
       'ethnicity_?', 'ethnicity_Asian', 'ethnicity_Black',
       'ethnicity_Hispanic', 'ethnicity_Latino', 'ethnicity_Others',
       'ethnicity_Pasifika', 'ethnicity_Turkish',
       'ethnicity_White-European', 'jundice_no', 'jundice_yes',
       'family_history_of_PDD_no', 'family_history_of_PDD_yes',
       "contry_of_res_'Costa Rica'", "contry_of_res_'Isle of Man'",
       "contry_of_res_'New Zealand'", "contry_of_res_'Saudi Arabia'",
       "contry_of_res_'South Africa'", "contry_of_res_'South Korea'",
       "contry_of_res_'U.S. Outlying Islands'",
       "contry_of_res_'United Arab Emirates'",
       "contry_of_res_'United Kingdom'", "contry_of_res_'United State

In [12]:
# print an example patient from the categorical data
X.loc[1]

A1_Score                               1
A2_Score                               1
A3_Score                               0
A4_Score                               0
A5_Score                               1
A6_Score                               1
A7_Score                               0
A8_Score                               1
A9_Score                               0
A10_Score                              0
age_10                                 0
age_11                                 0
age_4                                  0
age_5                                  0
age_6                                  1
age_7                                  0
age_8                                  0
age_9                                  0
age_?                                  0
gender_f                               0
gender_m                               1
ethnicity_'Middle Eastern '            1
ethnicity_'South Asian'                0
ethnicity_?                            0
ethnicity_Asian 

In [13]:
# convert the class data to categorical values - one-hot-encoded vectors
Y = pd.get_dummies(y)

In [14]:
Y.iloc[:10]

NO  YES
0   1    0
1   1    0
2   1    0
3   1    0
4   0    1
5   1    0
6   0    1
7   0    1
8   0    1
9   1    0

### 3. Split the Dataset into Training and Testing Datasets

Before we can begin training our neural network, we need to split the dataset into training and testing datasets. This will allow us to test our network after we are done training to determine how well it will generalize to new data. This step is incredibly easy when using the train_test_split() function provided by scikit-learn!

In [15]:
from sklearn import model_selection
# split the X and Y data into training and testing datasets
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size = 0.2)

In [16]:
print X_train.shape
print X_test.shape
print Y_train.shape
print Y_test.shape

(233, 96)
(59, 96)
(233, 2)
(59, 2)


### 4. Building the Network - Keras

In this project, we are going to use Keras to build and train our network. This model will be relatively simple and will only use dense (also known as fully connected) layers. This is the most common neural network layer. The network will have one hidden layer, use an Adam optimizer, and a categorical crossentropy loss. We won't worry about optimizing parameters such as learning rate, number of neurons in each layer, or activation functions in this project; however, if you have the time, manually adjusting these parameters and observing the results is a great way to learn about their function!

In [17]:
# build a neural network using Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# define a function to build the keras model
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(8, input_dim=96, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, kernel_initializer='normal', activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    
    # compile model
    adam = Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

model = create_model()

print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 8)                 776       
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 10        
Total params: 822
Trainable params: 822
Non-trainable params: 0
_________________________________________________________________
None


### 5. Training the Network

Now it's time for the fun! Training a Keras model is as simple as calling model.fit().

In [18]:
# fit the model to the training data
model.fit(X_train, Y_train, epochs=50, batch_size=10, verbose = 1)

Epoch 1/50
233/233 [==============================] - 0s 288us/step - loss: 0.6927 - acc: 0.5794
Epoch 2/50
233/233 [==============================] - 0s 245us/step - loss: 0.6910 - acc: 0.7210
Epoch 3/50
233/233 [==============================] - 0s 258us/step - loss: 0.6868 - acc: 0.7639
Epoch 4/50
233/233 [==============================] - 0s 236us/step - loss: 0.6779 - acc: 0.7082
Epoch 5/50
233/233 [==============================] - 0s 236us/step - loss: 0.6619 - acc: 0.8541
Epoch 6/50
233/233 [==============================] - 0s 305us/step - loss: 0.6340 - acc: 0.8283
Epoch 7/50
233/233 [==============================] - 0s 227us/step - loss: 0.5963 - acc: 0.8541
Epoch 8/50
233/233 [==============================] - 0s 305us/step - loss: 0.5446 - acc: 0.9399
Epoch 9/50
233/233 [==============================] - 0s 240us/step - loss: 0.4884 - acc: 0.8884
Epoch 10/50
233/233 [==============================] - 0s 227us/step - loss: 0.4220 - acc: 0.9227
Epoch 11/50
233/233 [========

### 6. Testing and Performance Metrics

Now that our model has been trained, we need to test its performance on the testing dataset. The model has never seen this information before; as a result, the testing dataset allows us to determine whether or not the model will be able to generalize to information that wasn't used during its training phase. We will use some of the metrics provided by scikit-learn for this purpose! 

In [19]:
# generate classification report using predictions for categorical model
from sklearn.metrics import classification_report, accuracy_score

predictions = model.predict_classes(X_test)
predictions

array([1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0], dtype=int64)

In [20]:
print('Results for Categorical Model')
print(accuracy_score(Y_test[['YES']], predictions))
print(classification_report(Y_test[['YES']], predictions))

Results for Categorical Model
0.9661016949152542
             precision    recall  f1-score   support

          0       0.97      0.97      0.97        36
          1       0.96      0.96      0.96        23

avg / total       0.97      0.97      0.97        59

